In [1]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt

In [2]:
# Load the data
df = pd.read_csv('cat_model_data.csv', index_col = 0)

In [ ]:
df.

MIS_Status               object
City                     object
State                    object
Zip                       int64
Bank                     object
BankState                object
NAICS                     int64
Term                      int64
NoEmp                     int64
NewExist                 object
CreateJob                 int64
RetainedJob               int64
FranchiseCode             int64
UrbanRural               object
RevLineCr                object
LowDoc                   object
GrAppv                    int64
NAICS_class_code          int64
Industry                 object
FranchiseCode_Encoded    object
RealEstate_Backed        object
Region                   object
TermDays                  int64
Recession                 int64
dtype: object

In [13]:
df['LowDoc'].value_counts(dropna=False)

LowDoc
N    783233
Y    113934
Name: count, dtype: int64

In [12]:
df.columns

Index(['MIS_Status', 'City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS',
       'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc', 'GrAppv',
       'NAICS_class_code', 'Industry', 'FranchiseCode_Encoded',
       'RealEstate_Backed', 'Region', 'TermDays', 'Recession'],
      dtype='object')

In [ ]:
df['MIS_Status'] = df['MIS_Status'].astype('str')
df['City'] = df['City'].astype('str')
df['State'] = df['State'].astype('str')
df['Bank'] = df['BankState'].astype('str')
df['BankState'] = df['BankState'].astype('str')
df['NewExist'] = df['NewExist'].astype('int64')
df['NewExist'] = df['NewExist'].astype('str')
df['UrbanRural'] = df['UrbanRural'].astype('str')
df['RevLineCr'] = df['RevLineCr'].astype('str')
df['LowDoc'] = df['LowDoc'].astype('str')
df['GrAppv'] = df['GrAppv'].astype('int64')
df['Industry'] = df['Industry'].astype('str')



In [11]:
pd.set_option('display.max_columns', None)

df.head()

,MIS_Status,City,State,Zip,Bank,BankState,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,GrAppv,NAICS_class_code,Industry,FranchiseCode_Encoded,RealEstate_Backed,Region,TermDays,Recession
0,PIF,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,84,4,2.0,0,0,1,NaN,N,Y,60000,45,Retail trade,No,No,Eastern,2520,0
1,PIF,NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,60,2,2.0,0,0,1,NaN,N,Y,40000,72,Accommodation and food services,No,No,Eastern,1800,0
2,PIF,BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,180,7,1.0,0,0,1,NaN,N,N,287000,62,Health care and social assistance,No,No,Eastern,5400,0
3,PIF,BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,60,2,1.0,0,0,1,NaN,N,Y,35000,81,Other services,No,No,Eastern,1800,0
4,PIF,ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,240,14,1.0,7,7,1,NaN,N,N,229000,81,Other services,No,Yes,Eastern,7200,0


In [6]:
df_cat_impute = df.copy()

In [ ]:
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler, PolynomialFeatures
# from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
# import lightgbm as lgb

# # Split the data into features and target
# X_lgb_rand = df_iter_impute.drop(columns=['MIS_Status_Encoded'])
# y_lgb_rand = df_iter_impute['MIS_Status_Encoded']

# # Split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X_lgb_rand, y_lgb_rand, test_size=0.2, random_state=42, stratify=y_lgb_rand)

# # Define numerical and categorical features
# numerical_features = ['GrAppv', 'CreateJob', 'Term', 'NoEmp', 'NAICS', 'NAICS_class_code']
# categorical_features = ['NewExist_Encode', 'UrbanRural_Encoded', 'LowDoc_Encoded', 'RevLineCr_Encoded', 
#                         'RevLineCr_Encoded_missing', 'FranchiseCode_Encoded', 'RealEstate_Backed', 'Region_Eastern',
#                         'Region_Northern', 'Region_Southern', 'Region_Western']

# # Define the preprocessor
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', StandardScaler(), numerical_features),
#         ('cat', 'passthrough', categorical_features)
#     ])

# # Build Pipeline
# model_pipeline = Pipeline([
#     ('preprocessor', preprocessor),
#     ('poly', PolynomialFeatures(degree=2)),
#     ('lgb', lgb.LGBMClassifier()),
# ])

# # Set RandomSearch parameters
# params = {
#     "lgb__boosting_type": ['gbdt', 'dart', 'rf'],
#     "lgb__learning_rate": [0.01, 0.05, 0.1],
#     "lgb__min_child_weight": [0.1],
#     "lgb__importance_type": ['gain'],
#     "lgb__objective": ['binary'],
#     "lgb__subsample": [1.0],
#     "lgb__n_estimators": [200],
#     "lgb__min_split_gain": [0.1],
#     "lgb__bagging_freq": [1],  # Ensure bagging_freq is greater than 0
#     "lgb__bagging_fraction": [0.8],  # Ensure bagging_fraction is between 0.0 and 1.0
#     "lgb__feature_fraction": [0.8]  # Ensure feature_fraction is between 0.0 and 1.0
# }

# # Set RandomSearchCV parameters
# lgb_grid_search = GridSearchCV(model_pipeline, param_grid=params, cv=5, error_score='raise', verbose=0)
# lgb_grid_search.fit(X_train, y_train)

# # Best parameterd and score
# print("Best Params", lgb_grid_search.best_estimator_)
# print("Best Score:", lgb_grid_search.best_score_)

In [7]:
df_cat_impute.columns

Index(['MIS_Status', 'City', 'State', 'Zip', 'Bank', 'BankState', 'NAICS',
       'Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob',
       'FranchiseCode', 'UrbanRural', 'RevLineCr', 'LowDoc', 'GrAppv',
       'NAICS_class_code', 'Industry', 'FranchiseCode_Encoded',
       'RealEstate_Backed', 'Region', 'TermDays', 'Recession'],
      dtype='object')

In [16]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

X = df_cat_impute.drop('MIS_Status', axis=1)
y = df_cat_impute['MIS_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = CatBoostClassifier(
    iterations=1000, 
    learning_rate=0.1, 
    depth=6, 
    verbose=100)

# Détecter automatiquement les  catégories
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Obtenir les indices de ces colonnes
categorical_features_indices = [X.columns.get_loc(col) for col in categorical_features]

# Créer les pools de données
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
test_pool = Pool(X_test, y_test, cat_features=categorical_features_indices)

model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=50)

# cat_boost_best = CatBoostClassifier(
#     task_type="GPU",  # Utilisation du GPU nvidia pour accélérer l'entraînement
#     random_state=42,
#     verbose=100,  # Affichage des logs toutes les 100 itérations
#     **cat_boost_params= cat_boost_params # Appliquer tes paramètres optimaux
# )
# cat_boost_params = {'scale_pos_weight': 1.0,
#  'random_strength': 10.0,
#  'learning_rate': 0.15,
#  'l2_leaf_reg': 5.0,
#  'iterations': 800.0,
#  'depth': 10.0,
#  'border_count': 128.0,
#  'bagging_temperature': 0.0}

# cat_boost_best.fit(X_train, y_train, cat_features=categorical_features)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [18]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

X = df_cat_impute.drop('MIS_Status', axis=1)
y = df_cat_impute['MIS_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

cat_boost_params = {
    'scale_pos_weight': 1.0,
    'random_strength': 10.0,
    'learning_rate': 0.15,
    'l2_leaf_reg': 5.0,
    'iterations': 800,
    'depth': 10,
    'border_count': 128,
    'bagging_temperature': 0.0
}

cat_boost_best = CatBoostClassifier(
    task_type="GPU",  # Use GPU for faster training if available
    random_state=42,
    verbose=100,  # Log every 100 iterations
    **cat_boost_params  # Apply your optimal parameters
)

# Detect categorical features
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Get the indices of these columns
categorical_features_indices = [X.columns.get_loc(col) for col in categorical_features]

# Create data pools
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
test_pool = Pool(X_test, y_test, cat_features=categorical_features_indices)

cat_boost_best.fit(train_pool, eval_set=test_pool, early_stopping_rounds=50)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject